In [ ]:
!git clone https://github.com/scikit-learn/sklearn-pypi-package

Cloning into 'sklearn-pypi-package'...
remote: Enumerating objects: 105, done.
remote: Counting objects: 100% (105/105), done.
remote: Compressing objects: 100% (60/60), done.
remote: Total 105 (delta 58), reused 81 (delta 44), pack-reused 0 (from 0)
Receiving objects: 100% (105/105), 18.46 KiB | 4.62 MiB/s, done.
Resolving deltas: 100% (58/58), done.


In [ ]:
import librosa
import soundfile
import os, glob, pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

In [ ]:
def extract_feature(file_name, mfcc, chroma, mel):
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate = sound_file.samplerate
        if chroma:
            stft = np.abs(librosa.stft(X))
        result = np.array([])
        if mfcc:
            mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            result = np.hstack((result, mfccs))
        if chroma:
            chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T, axis=0)
            result = np.hstack((result, chroma))
        if mel:
            mel = np.mean(librosa.feature.melspectrogram(y=X, sr=sample_rate).T, axis=0)
            result = np.hstack((result, mel))
        return result

In [ ]:

emotions={
  '01':'neutral',
  '02':'calm',
  '03':'happy',
  '04':'sad',
  '05':'angry',
  '06':'fearful',
  '07':'disgust',
  '08':'surprised'
}


observed_emotions=['calm', 'happy', 'fearful', 'disgust']

In [ ]:
def load_data(test_size=0.2):
    x, y = [], []
    file_count = 0


    for file in glob.glob("/content/drive/MyDrive/speech-emotion-recognition-ravdess-data/Actor_**/*.wav"):
        file_name = os.path.basename(file)
        emotion = emotions[file_name.split("-")[2]]

        if emotion not in observed_emotions:
            continue

        feature = extract_feature(file, mfcc=True, chroma=True, mel=True)

        if feature.size == 0:
            print(f"No features extracted for file: {file}")
            continue

        x.append(feature)
        y.append(emotion)
        file_count += 1


    print(f"Total files processed: {file_count}")

    if len(x) == 0:
        print("No data loaded, please check the file path or dataset.")
        return None

    return train_test_split(np.array(x), y, test_size=test_size, random_state=9)

In [ ]:
x_train,x_test,y_train,y_test=load_data(test_size=0.25)

Total files processed: 768


In [ ]:

print((x_train.shape[0], x_test.shape[0]))

(576, 192)


In [ ]:

print(f'Features extracted: {x_train.shape[1]}')

Features extracted: 180


In [ ]:

model=MLPClassifier(alpha=0.01, batch_size=256, epsilon=1e-08, hidden_layer_sizes=(300,), learning_rate='adaptive', max_iter=500)

In [ ]:

model.fit(x_train,y_train)

MLPClassifier(alpha=0.01, batch_size=256, hidden_layer_sizes=(300,),
              learning_rate='adaptive', max_iter=500)

In [ ]:

y_pred=model.predict(x_test)

In [ ]:

accuracy=accuracy_score(y_true=y_test, y_pred=y_pred)

print("Accuracy: {:.2f}%".format(accuracy*100))

Accuracy: 66.67%
